<a href="https://colab.research.google.com/github/Y4S1-Deep-Learning/Apple-Leaves-Desease-Detection---Supervised-Learning/blob/kaushalya/Mask_RCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing Mask RCNN**

In [ ]:
!git clone https://github.com/akTwelve/Mask_RCNN.git

In [ ]:
import os
import sys
os.chdir('Mask_RCNN/samples')

In [ ]:
ROOT_DIR = os.path.abspath("../")
sys.path.append(ROOT_DIR)

**Install libraries**

In [ ]:
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from Mask_RCNN.mrcnn.utils import Dataset
from matplotlib import pyplot
from Mask_RCNN.mrcnn.visualize import display_instances
from Mask_RCNN.mrcnn.utils import extract_bboxes
from Mask_RCNN.mrcnn.config import Config
from Mask_RCNN.mrcnn import utils

**Load training dataset**

In [ ]:
#images_path = "dataset/images.zip"
#annotation_path = "dataset/annots.zip"

#!unzip(os.path.join("content", images_path), "/content/dataset/train/images")
!unzip "/content/dataset/images.zip" -d "/content/dataset/train/"
!unzip "/content/dataset/annots.zip" -d "/content/dataset/train/"